## Data Cleaning

### Using scikitlearn for imputing missing values using different statistics/strategies for different columns

In [1]:
# importing libraries
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
# load dataset
train = pd.read_csv(r"C:\DS\house\train.csv")
test = pd.read_csv(r"C:\DS\house\test.csv")

In [3]:
# view shape
print('train dataset shape is ', train.shape)
print('test dataset shape is ', test.shape)

train dataset shape is  (1460, 81)
test dataset shape is  (1459, 80)


In [5]:
# 1 variable is less in test df because the target variable is not included - you can check this with head command
# getting x and y in training dataset
x_train = train.drop(columns = ['SalePrice'])
y_train = train['SalePrice']
x_test = test.copy()
print('x train dataset shape is ', x_train.shape)
print('y train dataset shape is ', y_train.shape)
print('x test dataset shape is ', x_test.shape)

x train dataset shape is  (1460, 80)
y train dataset shape is  (1460,)
x test dataset shape is  (1459, 80)


In [29]:
# getting missing variables cat and num
catvars = x_train.select_dtypes(include = ['object'])
catvarscol = catvars.isnull().sum() > 0
catvarsmiss = catvarscol[catvarscol == True].keys()
catvarsmiss

Index(['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu',
       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC',
       'Fence', 'MiscFeature'],
      dtype='object')

In [30]:
# getting missing variables cat and num
numvars = x_train.select_dtypes(include = ['float64', 'int64'])
numvarscol = numvars.isnull().sum() > 0
numvarsmiss = numvarscol[numvarscol == True].keys()
numvarsmiss

Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object')

### we will use pipeline

In [33]:
# we want mean in following num var
numvar_mean = ['LotFrontage']
# we want median in following num vars
numvar_median = ['MasVnrArea', 'GarageYrBlt']
# we want most frequent value in following cat vars
catvars_mode = ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu']
# we want a constant value in following cat vars
catvars_constant = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC',
       'Fence', 'MiscFeature']

In [34]:
# pipeline defining
numvar_mean_imputer = Pipeline(steps = [("imputer", SimpleImputer(strategy = 'mean'))])
numvar_median_imputer = Pipeline(steps = [("imputer", SimpleImputer(strategy = 'median'))])
catvars_mode_imputer = Pipeline(steps = [("imputer", SimpleImputer(strategy = 'most_frequent'))])
catvars_constant_imputer = Pipeline(steps = [("imputer", SimpleImputer(strategy = 'constant', fill_value = 'missing'))])

In [35]:
# now we have to apply different strategies defined using pipeline above to different columns
# we use ColumnTransformer
preprocessor = ColumnTransformer(transformers = [('mean_imputer', numvar_mean_imputer, numvar_mean),
                                 ('median_imputer', numvar_median_imputer, numvar_median),
                                 ('mode_imputer', catvars_mode_imputer, catvars_mode), 
                                 ('constant_imputer', catvars_constant_imputer, catvars_constant)])

In [36]:
# fitting the defined transformation
preprocessor.fit(x_train)

ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('constant_imputer',
                                 Pip

In [40]:
# to check what method was applied to which columns do this
preprocessor.transform

<bound method ColumnTransformer.transform of ColumnTransformer(transformers=[('mean_imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode_imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2', 'Electrical',
                                  'FireplaceQu']),
                                ('constant_i

In [41]:
# to check what values were selected for what columns
preprocessor.named_transformers_['mean_imputer'].named_steps['imputer'].statistics_

array([70.04995837])

In [43]:
# checking if this is ok
train['LotFrontage'].mean()

70.04995836802665

In [44]:
# it is correct
# checking median
preprocessor.named_transformers_['median_imputer'].named_steps['imputer'].statistics_

array([   0., 1980.])

In [45]:
# checking mode
preprocessor.named_transformers_['mode_imputer'].named_steps['imputer'].statistics_

array(['Grvl', 'None', 'TA', 'TA', 'No', 'Unf', 'Unf', 'SBrkr', 'Gd'],
      dtype=object)

In [46]:
# checking constant
preprocessor.named_transformers_['constant_imputer'].named_steps['imputer'].statistics_

array(['missing', 'missing', 'missing', 'missing', 'missing', 'missing',
       'missing'], dtype=object)

In [47]:
# now let us apply these selected values in our data
x_train_clean = preprocessor.transform(x_train)
x_test_clean = preprocessor.transform(x_test)

In [48]:
x_train_clean

array([[65.0, 196.0, 2003.0, ..., 'missing', 'missing', 'missing'],
       [80.0, 0.0, 1976.0, ..., 'missing', 'missing', 'missing'],
       [68.0, 162.0, 2001.0, ..., 'missing', 'missing', 'missing'],
       ...,
       [66.0, 0.0, 1941.0, ..., 'missing', 'GdPrv', 'Shed'],
       [68.0, 0.0, 1950.0, ..., 'missing', 'missing', 'missing'],
       [75.0, 0.0, 1965.0, ..., 'missing', 'missing', 'missing']],
      dtype=object)

In [51]:
# it is 2d array - convert to df
# first check remainder
preprocessor.transformers_

[('mean_imputer',
  Pipeline(steps=[('imputer', SimpleImputer())]),
  ['LotFrontage']),
 ('median_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))]),
  ['MasVnrArea', 'GarageYrBlt']),
 ('mode_imputer',
  Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent'))]),
  ['Alley',
   'MasVnrType',
   'BsmtQual',
   'BsmtCond',
   'BsmtExposure',
   'BsmtFinType1',
   'BsmtFinType2',
   'Electrical',
   'FireplaceQu']),
 ('constant_imputer',
  Pipeline(steps=[('imputer',
                   SimpleImputer(fill_value='missing', strategy='constant'))]),
  ['GarageType',
   'GarageFinish',
   'GarageQual',
   'GarageCond',
   'PoolQC',
   'Fence',
   'MiscFeature']),
 ('remainder',
  'drop',
  [0,
   1,
   2,
   4,
   5,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   27,
   28,
   29,
   34,
   36,
   37,
   38,
   39,
   40,
   41,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
  

In [52]:
# the nunmbers gives the index of columns where no missing values were found and thus were dropped
# if we want not to drop these columns, we have to give a value to remainder - passthrough
# we can do that using remainder function later
# now let us convert to df
xtrain_clean_miss = pd.DataFrame(x_train_clean, columns = numvar_mean+numvar_median+catvars_mode+catvars_constant)
# note that the columns should be defined in the same order as defined earlier


In [53]:
xtrain_clean_miss.head()

,LotFrontage,MasVnrArea,GarageYrBlt,Alley,MasVnrType,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Electrical,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MiscFeature
0,65.0,196.0,2003.0,Grvl,BrkFace,Gd,TA,No,GLQ,Unf,SBrkr,Gd,Attchd,RFn,TA,TA,missing,missing,missing
1,80.0,0.0,1976.0,Grvl,None,Gd,TA,Gd,ALQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing
2,68.0,162.0,2001.0,Grvl,BrkFace,Gd,TA,Mn,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing
3,60.0,0.0,1998.0,Grvl,None,TA,Gd,No,ALQ,Unf,SBrkr,Gd,Detchd,Unf,TA,TA,missing,missing,missing
4,84.0,350.0,2000.0,Grvl,BrkFace,Gd,TA,Av,GLQ,Unf,SBrkr,TA,Attchd,RFn,TA,TA,missing,missing,missing


In [54]:
xtrain_clean_miss.isnull().sum().sum()

0

In [55]:
# checking how the values changed for example in Alley
train["Alley"].value_counts()

Grvl    50
Pave    41
Name: Alley, dtype: int64

In [56]:
xtrain_clean_miss["Alley"].value_counts()

Grvl    1419
Pave      41
Name: Alley, dtype: int64

In [57]:
# DRAWBACK: only those columns we get which are cleaned
# let us join other columns that did not need cleaning with this df
# check shape
xtrain_clean_miss.shape

(1460, 19)

In [60]:
x_train.shape

(1460, 61)

In [59]:
# dropping columns in original df which are included in clean df
x_train.drop(columns = xtrain_clean_miss.keys(), axis=1, inplace = True)

C:\Users\jaska\AppData\Local\Temp\ipykernel_12496\3037611833.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  xtrain_clean = x_train.append(xtrain_clean_miss)


In [65]:
x_train.shape

(1460, 61)

In [66]:
xtrain_clean = x_train.join(xtrain_clean_miss)

In [67]:
xtrain_clean.shape

(1460, 80)

In [71]:
xtrain_clean.isnull().sum().sum()

0

### train is now clean - the whole procedure can be repeated for test dataset as well